In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install transformers[torch] accelerate -U datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [12]:
!pip install torch transformers

In [14]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive

In [5]:
import os

data_path = r'C:\Users\Oh\final project\movie dialogue'

if not os.path.exists(data_path):
    print(f"The directory {data_path} does not exist.")
else:
    print(f"The directory {data_path} exists.")

The directory C:\Users\Oh\final project\movie dialogue does not exist.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
import json
from transformers import AutoTokenizer

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/movie dialogue'

# 모든 JSON 파일의 경로를 리스트로 저장
json_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]

# 데이터를 저장할 리스트
all_texts = []

# 각 JSON 파일을 불러와서 데이터 추가
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['info']:
            text = item['annotations']['text']
            all_texts.append(text)

print(f'Loaded {len(all_texts)} dialogues.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 4276 dialogues.


In [15]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# 텍스트 데이터를 추출하여 토큰화
tokenized_texts = [tokenizer(text, truncation=True, padding='max_length', max_length=512) for text in all_texts]

# 토큰화된 데이터를 저장할 리스트
input_ids = [tokens['input_ids'] for tokens in tokenized_texts]
attention_masks = [tokens['attention_mask'] for tokens in tokenized_texts]

# 데이터셋 생성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_masks[idx])
        }

dataset = CustomDataset(input_ids, attention_masks)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [21]:
!pip install torch transformers

In [48]:
import os
import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/movie dialogue'

# 모든 JSON 파일의 경로를 리스트로 저장
json_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]

# 데이터를 저장할 리스트
all_texts = []

# 각 JSON 파일을 불러와서 데이터 추가
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['info']:
            text = item['annotations']['text']
            all_texts.append(text)

print(f'Loaded {len(all_texts)} dialogues.')

# 텍스트 데이터를 결합하여 GPT 모델 입력용 텍스트 생성
dataset_text = "\n".join(all_texts)

# 토크나이저 설정
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# 토큰화된 데이터 생성
inputs = tokenizer(dataset_text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs['input_ids']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.input_ids[idx]
        }

dataset = CustomDataset(inputs)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 4276 dialogues.


In [49]:
# 모델 설정
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    prediction_loss_only=True
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# 모델 학습
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=2.6267995834350586, metrics={'train_runtime': 11.6385, 'train_samples_per_second': 0.258, 'train_steps_per_second': 0.258, 'total_flos': 783876096000.0, 'train_loss': 2.6267995834350586, 'epoch': 3.0})

In [50]:
# 챗봇 응답 생성 함수
def generate_response(model, tokenizer, input_text, max_length=50):
    model.eval()
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



In [51]:
# 예시 입력 텍스트
input_text = "어떤 영화를 추천해줄래?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Response: 어떤 영화를 추천해줄래?

어떤 영화를 추


In [27]:
# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [38]:
# 모델 설정
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    prediction_loss_only=True,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# 모델 학습
trainer.train()

# 챗봇 응답 생성 함수
def generate_response(model, tokenizer, input_text, max_length=50):
    model.eval()
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 예시 입력 텍스트
input_text = "어떤 영화를 추천해줄래?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Step,Training Loss
10,2.591900
20,2.364700
30,2.080700
40,1.754100
50,1.406700
60,1.003400
70,0.642700
80,0.364100
90,0.197300
100,0.122600


Response: 어떤 영화를 추천해줄래?
어떤 잭! 아니지�


In [33]:
len(json_files)

4276

In [35]:
len(all_texts)

4276

In [52]:
!pip install torch transformers

In [57]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 경로 설정 (구글 드라이브 내의 경로)
data_path = '/content/drive/MyDrive/movie dialogue'

# 모든 JSON 파일의 경로를 리스트로 저장
json_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]

# 데이터를 저장할 리스트
dialogue_pairs = []

# 각 JSON 파일을 불러와서 데이터 추가
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data['info']:
            text = item['annotations']['text']
            # 대화 쌍 추출 (여기서는 간단히 첫 번째 줄과 두 번째 줄을 대화 쌍으로 가정)
            lines = text.split('\n')
            if len(lines) >= 2:
                dialogue_pairs.append((lines[0], lines[1]))

print(f'Loaded {len(dialogue_pairs)} dialogue pairs.')

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained('t5-small')

# 대화 데이터를 추출하여 토큰화
input_texts = [pair[0] for pair in dialogue_pairs]
target_texts = [pair[1] for pair in dialogue_pairs]

# 토큰화된 데이터를 저장할 리스트
inputs = tokenizer(input_texts, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
targets = tokenizer(target_texts, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

# 데이터셋 생성
class DialogueDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

dataset = DialogueDataset(inputs, targets)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 4276 dialogue pairs.


In [59]:
# 모델 설정
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

# 훈련 전 캐시 초기화
torch.cuda.empty_cache()

# 훈련 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # 배치 크기를 줄임
    per_device_eval_batch_size=8,   # 배치 크기를 줄임
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=4,  # Gradient Accumulation 사용
    fp16=True  # Mixed Precision Training 사용
)

# Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# 모델 학습
trainer.train()

# 훈련 후 캐시 초기화
torch.cuda.empty_cache()

Step,Training Loss
10,15.419900
20,15.656500
30,15.454400
40,15.225600
50,13.983400
60,13.101200
70,12.100000
80,10.330800
90,8.197200
100,6.347500


In [62]:
def generate_response(model, tokenizer, input_text):
    # 모델이 사용 중인 장치 가져오기
    device = next(model.parameters()).device

    # 입력 텍스트를 토큰화하고 텐서를 모델이 위치한 장치로 이동
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).to(device)

    # 응답 생성
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=50)

    # 출력 텐서를 디코딩하여 응답 생성
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 예시 입력 텍스트
input_text = "어떤 영화를 추천해줄래?"

# 모델을 사용하여 응답 생성
response = generate_response(model, tokenizer, input_text)
print(f"Response: {response}")

Response:     


In [ ]:
#추천 하는건 머신러닝이고
# 대화를 하는 건 ...